# COVID 19 Basic Visualization

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
# from bqplot import pyplot as plt
from IPython.display import clear_output

from ipywidgets import interact, Layout
import ipywidgets as widgets
import matplotlib.dates as mdates
import datetime as dt

import calmap

In [2]:
df_confirmed = pd.read_csv(r'data/time_series_covid19_confirmed_global.csv')
df_confirmed = df_confirmed.groupby(['Country/Region'], as_index=False).sum()
# df_confirmed.head()

In [3]:
df_deaths = pd.read_csv(r'data/time_series_covid19_deaths_global.csv')
df_deaths = df_deaths.groupby(['Country/Region'], as_index=False).sum()
# df_deaths.head()

In [4]:
df_recovered = pd.read_csv(r'data/time_series_covid19_recovered_global.csv')
df_recovered = df_recovered.groupby(['Country/Region'], as_index=False).sum()
# df_recovered.head()

In [59]:
import plotly.graph_objects as go

def f(x):
#     print(date_sel.value)
    dates = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in list(df_confirmed.columns[3:])]
#     print(dates.index(date_sel.value[0]), dates.index(date_sel.value[1]))
    st_val = dates.index(date_sel.value[0]) + 3
    end_val = dates.index(date_sel.value[1])
    
    x = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in list(df_confirmed.columns[st_val:end_val])]    
    y1 = df_confirmed[df_confirmed['Country/Region'] == country_sel.value].iloc[0][st_val:end_val]
    y1.index = pd.to_datetime(y1.index)
    y2 = df_deaths[df_deaths['Country/Region'] == country_sel.value].iloc[0][st_val:end_val]
    y2.index = pd.to_datetime(y1.index)
    y3 = df_recovered[df_recovered['Country/Region'] == country_sel.value].iloc[0][st_val:end_val]
    y3.index = pd.to_datetime(y1.index)
#     with fig1.batch_update():

    if confirmed_sel.value:
        fig1.data[0].x = y1.index
        fig1.data[0].y = y1
    else:
        fig1.data[0].x = None
        fig1.data[0].y = None
        
    if death_sel.value:
        fig1.data[1].x = y2.index
        fig1.data[1].y = y2
    else:
        fig1.data[1].x = None
        fig1.data[1].y = None
        
    if recover_sel.value:
        fig1.data[2].x = y3.index
        fig1.data[2].y = y3
    else:
        fig1.data[2].x = None
        fig1.data[2].y = None

    
    
country_sel = widgets.Dropdown(
    options=list(df_confirmed['Country/Region'].unique()),
    description='Country:',
    value='India',
    disabled=False,
)

confirmed_sel = widgets.Checkbox(
    value=True,
    description='Confirmed',
    disabled=False
)

death_sel = widgets.Checkbox(
    value=True,
    description='Death',
    disabled=False
)

recover_sel = widgets.Checkbox(
    value=True,
    description='Recovered',
    disabled=False
)

dates = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in list(df_confirmed.columns[3:])]
dates_i = range(len(dates))
options = [(i.strftime(' %d %b %y '), i) for i in dates]
date_sel = widgets.SelectionRangeSlider(
    options=options,
    index=(0, len(dates)-1),
    description='Months',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='70%', height='80px')
)
    
country_sel.observe(f, names='value')
confirmed_sel.observe(f, names='value')
death_sel.observe(f, names='value')
recover_sel.observe(f, names='value')
date_sel.observe(f, names='value')


x = [dt.datetime.strptime(d,'%m/%d/%y').date() for d in list(df_confirmed.columns[3:])]
    
y1 = df_confirmed[df_confirmed['Country/Region'] == 'India'].iloc[0][3:]
y1.index = pd.to_datetime(y1.index)
y2 = df_deaths[df_deaths['Country/Region'] == 'India'].iloc[0][3:]
y2.index = pd.to_datetime(y2.index)
y3 = df_recovered[df_recovered['Country/Region'] == 'India'].iloc[0][3:]
y3.index = pd.to_datetime(y3.index)

fig1 = go.FigureWidget(data=[
    go.Scatter(
        x=y1.index, y=y1,
        line=dict(color='blue', width=2),
        name="Confirmed"
    ),
    go.Scatter(
        x=y2.index, y=y2,
        line=dict(color='red', width=2),
        name="Deaths"
    ),
    go.Scatter(
        x=y3.index, y=y3,
        line=dict(color='green', width=2),
        name="Recovered"
    )
])
fig1.update_layout(
    title='Cumulative data',
    yaxis_title="Cases",
    xaxis_title="Time",
#     paper_bgcolor='rgba(0,0,0,0)',
#     plot_bgcolor='rgba(0,0,0,0)'
)


sel = container2 = widgets.HBox([country_sel, confirmed_sel, death_sel, recover_sel])
wid_all = widgets.VBox([sel,
              date_sel,
              fig1])
# display(wid_all)

In [61]:
import ipyvuetify as v
v.theme.dark = True

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Cumulative data']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg12=True, sm12=True, md12=True, x12=True, children=[
                country_sel, confirmed_sel, death_sel, recover_sel, date_sel
            ])
        ]),
        v.Layout(col=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg12=True, sm12=True, md12=True, x12=True, children=[
                fig1
            ])
        ])
    ]),
])

Tabs(children=[Tab(children=['Cumulative data']), TabItem(children=[Layout(align_center=True, children=[Flex(c…